In [ ]:
# Required Libraries
import datetime
from datetime import datetime
import time
from fb_duckling import Duckling
import spacy
import pprint


nlp = spacy.load('en_core_web_sm')
duckling = Duckling(locale="en_US")

In [ ]:
def get_ner(usr_input):
    
    # Creates a result dictionary for output
    result = {'Dimension': []}
    
    # Passing user input through SPACY, which tags with GPE(Location), ORG(Organisation), Person etc.
    usr_input = (usr_input)
    usr_input_doc = nlp(usr_input)  
    for ent in usr_input_doc.ents:
        if ent.label_ == 'GPE' or ent.label_ == 'ORG':
            result['Dimension'].append({ent.label_:ent.text})
    
    
    # Passing user input through Duckline, which tages with, date time, temperature, duration, distance etc.
    duck = duckling(usr_input)
    for d in duck:
        date = []
        time = []
        if d['dim'] == 'time':
            if 'type' in d['value'] and d['value']['type'] == 'interval':
                
                y = d['value']['from']['value'].split('.')[0]
                y = datetime.strptime(y, "%Y-%m-%dT%H:%M:%S" )
                date.append({'from': y.strftime("%m/%d/%Y")})
                time.append({'from': y.strftime("%H:%M:%S")})
                
                z = d['value']['to']['value'].split('.')[0]
                z = datetime.strptime(z, "%Y-%m-%dT%H:%M:%S" )
                date.append({'to': z.strftime("%m/%d/%Y")})
                time.append({'to': z.strftime("%H:%M:%S")})
                
            else:
                x = d['value']['value'].split('.')[0]
                x = datetime.strptime(x, "%Y-%m-%dT%H:%M:%S" )
                date.append({d['body']: x.strftime("%m/%d/%Y")})
                time.append({d['body']: x.strftime("%H:%M:%S")})
                
            now = datetime.now()
            for t in time:
                for key, value in t.items():
                    if value == '00:00:00':
                        t[key] = now.strftime('%H:%M %p')
                
            result['Dimension'].append({"Date": date})
            result['Dimension'].append({"Time": time})
            
        else:
            result['Dimension'].append({d['dim']:{d['body']:d['value']}})
    
    return result

In [ ]:
if __name__ == '__main__':

    usr_input = " "
    while usr_input.lower().strip()!= "quit":

        # Takes user input
        usr_input = input("-> Please enter your input: ")
        if usr_input == "":
            print("-> Not a valid string !!\n")
            continue
        # usr_input = "book a table for today at 10 in morning"
        # usr_input = "I will be in Bangalore at 5pm"

        # removes unwanted spaces in string
        usr_input = " ".join(usr_input.split())
        usr_input = usr_input.replace(",", "")
        usr_input = usr_input.replace(".", "")
        result = get_ner(usr_input)

        pp = pprint.PrettyPrinter(indent=4)
        
        if result["Dimension"]:
            # Print Output
            # print("\n")
            pp.pprint(result['Dimension'])

        else:
            print(result)
            print("No entity found, please try some other sentence.")
            print("\n\n")
        usr_input = input("-> Type 'quit' to exit or press enter to continue ... ")

    print("Thank You :-)")
    
    
    
# Example Input: I am going to Kormangala market tomorrow morning
#Output: 
# [   {'GPE': 'Kormangala'},
#     {'Date': [{'from': '08/15/2020'}, {'to': '08/15/2020'}]},
#     {'Time': [{'from': '00:00:00'}, {'to': '12:00:00'}]}]

# Example Input: I will be in bangalore between 6th to 7th may
# Output:
# [   {'Date': [{'from': '05/06/2021'}, {'to': '05/08/2021'}]},
#     {'Time': [{'from': '00:00:00'}, {'to': '00:00:00'}]}]


#Example Input: I will be in bangalore between 6th May 6 PM to 7th may 8 AM
# Output:
#     [   {'Date': [{'from': '05/06/2021'}, {'to': '05/07/2021'}]},
#     {'Time': [{'from': '18:00:00'}, {'to': '09:00:00'}]}]

#Example Input: 